In [1]:
import pandas as pd
from typing import List
import glob
import os
from tqdm import tqdm

In [2]:
NSW_PROPERTY_DATA_OUTPUT = "./output/nsw_property_sales_data.csv"
NSW_PROPERTY_ARCHIVED_DATA_OUTPUT = "./output/nsw_property_sales_archived_data.csv"
NSW_LAND_VALUE_DATA_OUTPUT = "./output/LV_20240301.csv"
NSW_VALUE_GENERAL_FILE_PATH = "./data/NSW_Property_Sales_Data"

propertySales_df = pd.read_csv(NSW_PROPERTY_DATA_OUTPUT, dtype={"property_id": "string"})

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [3]:
propertySales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4861243 entries, 0 to 4861242
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   property_id         string 
 1   download_date       object 
 2   council_name        object 
 3   purchase_price      float64
 4   address             object 
 5   post_code           float64
 6   property_type       object 
 7   strata_lot_number   float64
 8   property_name       object 
 9   area                float64
 10  area_type           object 
 11  contract_date       object 
 12  settlement_date     object 
 13  zoning              object 
 14  nature_of_property  object 
 15  primary_purpose     object 
 16  legal_description   object 
 17  dealing_number      object 
dtypes: float64(4), object(13), string(1)
memory usage: 667.6+ MB


In [4]:
propertySales_df.nunique()

property_id           1946503
download_date             989
council_name              220
purchase_price         100114
address               2647934
post_code                 709
property_type               2
strata_lot_number        1169
property_name           30622
area                    76909
area_type                   2
contract_date           12939
settlement_date         10409
zoning                     70
nature_of_property          3
primary_purpose         11206
legal_description     3156887
dealing_number        4485183
dtype: int64

In [5]:
def findNoUnit(row: pd.Series):
    if "No Unit" in row['address']:
        return True
    else:
        return False


In [6]:
noUnits_df = propertySales_df.apply(findNoUnit, axis=1)

In [7]:
propertySales_df["AddressConfigured"] = False
propertySales_df["noUnits"] = noUnits_df

In [9]:
propertySales_df

,property_id,download_date,council_name,purchase_price,address,post_code,property_type,strata_lot_number,property_name,area,area_type,contract_date,settlement_date,zoning,nature_of_property,primary_purpose,legal_description,dealing_number,AddressConfigured,noUnits
0,4058750,2024-02-19,NARRABRI,3186600.0,"19 KAMILAROI HWY, NARRABRI",2390.0,house,NaN,NaN,4.782,H,2022-09-09,2024-02-07,RU1,V,VACANT LAND,2/1222972,AT820257,False,False
1,4124909,2024-02-19,NARRABRI,3186600.0,"12169 NEWELL HWY, NARRABRI",2390.0,house,NaN,NaN,1215.000,M,2022-09-09,2024-02-07,E4,V,VACANT LAND,32/1184780,AT820257,False,False
2,4531338,2024-02-19,NARRABRI,11330.0,"No Unit or Housing Number NEWELL HWY, NARRABRI",2390.0,house,NaN,NaN,1.342,H,2022-08-09,2024-02-07,NaN,V,VACANT LAND,2/1288400,AT821191,False,True
3,4531644,2024-02-19,NARRABRI,3186600.0,"No Unit or Housing Number KAMILAROI HWY, NARRABRI",2390.0,house,NaN,NaN,129.100,H,2022-09-09,2024-02-07,NaN,V,VACANT LAND,4/1036726,AT820257,False,True
4,2656759,2024-02-19,NARRABRI,220000.0,"82 BOUNDARY ST, WEE WAA",2388.0,house,NaN,NaN,992.700,M,2024-02-06,2024-02-09,R1,R,RESIDENCE,7/230595,AT824385,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861238,3137396,2004-12-16,WOLLONGONG,190000.0,"1/186 BALGOWNIE RD, BALGOWNIE",2519.0,unit,1.0,NaN,NaN,NaN,2004-05-27,2004-11-29,NaN,R,RESIDENCE,1/SP73961,AB141473,False,False
4861239,3137396,2004-12-16,WOLLONGONG,190000.0,"2/186 BALGOWNIE RD, BALGOWNIE",2519.0,unit,2.0,NaN,NaN,NaN,2004-05-27,2004-11-29,NaN,R,RESIDENCE,2/SP73961,AB141473,False,False
4861240,3138813,2004-12-16,WOLLONGONG,380000.0,"No Unit or Housing Number HAWTHORN ST, TARRAWANNA",2518.0,house,NaN,NaN,21.170,H,2003-12-15,2004-06-25,NaN,V,VACANT LAND,2/793302,AA769421,False,True
4861241,3138824,2004-12-16,WOLLONGONG,5500.0,"No Unit or Housing Number HOBART ST, BULLI",2516.0,house,NaN,NaN,43.360,H,2003-06-10,2003-06-10,NaN,V,VACANT LAND,5/255282,9686125,False,True


In [32]:
noUnitsProperty_df = propertySales_df[propertySales_df["noUnits"] == True]
noUnitsProperty_df

,property_id,download_date,council_name,purchase_price,address,post_code,property_type,strata_lot_number,property_name,area,area_type,contract_date,settlement_date,zoning,nature_of_property,primary_purpose,legal_description,dealing_number,AddressConfigured,noUnits
2,4531338,2024-02-19,NARRABRI,11330.0,"No Unit or Housing Number NEWELL HWY, NARRABRI",2390.0,house,NaN,NaN,1.342,H,2022-08-09,2024-02-07,NaN,V,VACANT LAND,2/1288400,AT821191,False,True
3,4531644,2024-02-19,NARRABRI,3186600.0,"No Unit or Housing Number KAMILAROI HWY, NARRABRI",2390.0,house,NaN,NaN,129.100,H,2022-09-09,2024-02-07,NaN,V,VACANT LAND,4/1036726,AT820257,False,True
96,4530671,2024-02-19,THE HILLS SHIRE,2500000.0,"No Unit or Housing Number BIRDS LANE, MARAYLYA",2765.0,house,NaN,NaN,4.462,H,2023-10-23,2024-01-23,NaN,V,VACANT LAND,2/874915,AT784346,False,True
146,4528579,2024-02-19,DUNGOG,675000.0,"No Unit or Housing Number GLENDONBROOK RD, GRE...",2311.0,house,NaN,NaN,22.900,H,2023-12-19,2024-01-19,NaN,R,RESIDENCE,OF THE PART FORMERLY IN 60/1225835,AT786494,False,True
168,4531634,2024-02-19,WOOLLAHRA,247877.0,"No Unit or Housing Number WUNULLA RD, POINT PIPER",2027.0,house,NaN,NaN,NaN,NaN,2021-07-28,2022-09-19,NaN,3,ROAD RESERVE,PT 10/1291286,AT798277,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861231,3132149,2004-12-16,WOLLONGONG,220000.0,"No Unit or Housing Number RICKARD RD, UNANDERRA",2526.0,house,NaN,NaN,460.900,M,2004-07-06,2004-11-25,A,V,VACANT LAND,33/1075053,AB155826,False,True
4861236,3135607,2004-12-16,WOLLONGONG,490000.0,"No Unit or Housing Number HOLLYMOUNT VIEW, WOO...",2517.0,house,NaN,NaN,677.300,M,2004-06-09,2004-12-01,NaN,V,VACANT LAND,801/1069433,AB141407,False,True
4861237,3135609,2004-12-16,WOLLONGONG,495000.0,"No Unit or Housing Number HOLLYMOUNT VIEW, WOO...",2517.0,house,NaN,NaN,1414.000,M,2004-07-26,2004-12-07,NaN,V,VACANT LAND,803/1069433,AB152536,False,True
4861240,3138813,2004-12-16,WOLLONGONG,380000.0,"No Unit or Housing Number HAWTHORN ST, TARRAWANNA",2518.0,house,NaN,NaN,21.170,H,2003-12-15,2004-06-25,NaN,V,VACANT LAND,2/793302,AA769421,False,True


In [45]:
landValue_df = pd.read_csv(NSW_LAND_VALUE_DATA_OUTPUT, lineterminator='\n', dtype={"PROPERTY ID": "string"})

In [46]:
landValue_df

,PROPERTY ID,PROPERTY DESCRIPTION,ADDRESS,POSTCODE,DISTRICT NAME,DISTRICT CODE,LAND VALUE
0,1213489,4/877506,"162 GARRA RD, MOLONG",2866.0,CABONNE,109,"{'2023': '2460000', '2022': '2140000', '2021':..."
1,1213490,70/790734,"76 SPRING VALE LANE, MOLONG",2866.0,CABONNE,109,"{'2023': '2830000', '2022': '2460000', '2021':..."
2,1213492,179/750133,"179 LARRAS LEE RD, LARRAS LEE",2866.0,CABONNE,109,"{'2023': '65600', '2022': '50900', '2021': '40..."
3,1213493,"181, 202/750133","22 BLOOMFIELD RD, MOLONG",2866.0,CABONNE,109,"{'2023': '638000', '2022': '495000', '2021': '..."
4,1213494,182/750133,"35 NYORA LANE, MOLONG",2866.0,CABONNE,109,"{'2023': '638000', '2022': '495000', '2021': '..."
...,...,...,...,...,...,...,...
2692604,4304538,12/756250,"60 MILLS RD, BIRGANBIGIL",2710.0,EDWARD RIVER,262,"{'2023': '807000', '2022': '708000', '2021': '..."
2692605,4310219,"1/541199 46, 64, 75/756274 1/1263838","370 HOLSCHIERS RD, PRETTY PINE",2710.0,EDWARD RIVER,262,"{'2023': '1580000', '2022': '1510000', '2021':..."
2692606,4287134,"1/48568 12, 13, 19, 20, 21, 24, 25/756291 1, 2...","MABINS WELL RD, MABINS WELL",2716.0,EDWARD RIVER,262,"{'2023': '3670000', '2022': '3300000', '2021':..."
2692607,4287137,"1/102095 4, 7, 12, 28, 50, 57, 58, 124, 125, 1...","4913 CONARGO RD, CONARGO",2710.0,EDWARD RIVER,262,"{'2023': '8000000', '2022': '5280000', '2021':..."


In [47]:
landValue_df["PROPERTY ID"]

0          1213489
1          1213490
2          1213492
3          1213493
4          1213494
            ...   
2692604    4304538
2692605    4310219
2692606    4287134
2692607    4287137
2692608    4287144
Name: PROPERTY ID, Length: 2692609, dtype: string

In [50]:
def linkPropLandValue(row: pd.Series):
    landvalueProp = landValue_df[landValue_df["PROPERTY ID"] == row["property_id"]]
    if len(landvalueProp) > 0:
        return landvalueProp["ADDRESS"]
    else:
        return ""

In [51]:
addressFilled_df = noUnitsProperty_df.apply(linkPropLandValue, axis=1)

In [36]:
noUnitsProperty_df["addressFilled"] = addressFilled_df

/var/folders/hx/4vzr554n7m5b4tw1nr766g_m0000gn/T/ipykernel_32384/2596283966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noUnitsProperty_df["addressFilled"] = addressFilled_df


0          1213489
1          1213490
2          1213492
3          1213493
4          1213494
            ...   
2692604    4304538
2692605    4310219
2692606    4287134
2692607    4287137
2692608    4287144
Name: PROPERTY ID, Length: 2692609, dtype: int64